## Load Libraries

In [1]:
import pandas as pd
import numpy as np

## Load total CSV after downloading from STRAVA API

In [2]:
# Load it into a Dataframe using pandas
path = 'RideData_20170508211642.csv'
df = pd.read_csv(path)
df.head()

,act_startDate,timestamp,act_id,act_name,altitude,distance,grade_smooth,heartrate,moving,time,velocity_smooth,lat,long
0,2017-03-27 06:52:47+00:00,2017-03-27 06:52:47+00:00,916298883,Morning Ride,8.0,0.0,3.2,NaN,False,0,0.0,55.674874,12.592714
1,2017-03-27 06:52:47+00:00,2017-03-27 06:52:57+00:00,916298883,Morning Ride,8.0,4.4,5.1,NaN,True,10,0.4,55.674910,12.592685
2,2017-03-27 06:52:47+00:00,2017-03-27 06:52:59+00:00,916298883,Morning Ride,8.3,9.3,5.4,NaN,True,12,0.8,55.674938,12.592625
3,2017-03-27 06:52:47+00:00,2017-03-27 06:53:01+00:00,916298883,Morning Ride,8.8,15.6,6.3,NaN,True,14,1.1,55.674979,12.592555
4,2017-03-27 06:52:47+00:00,2017-03-27 06:53:02+00:00,916298883,Morning Ride,9.0,18.6,6.0,NaN,True,15,2.8,55.674997,12.592519


## Useful Functions

In [3]:
# Trim data per specific time threshold
def trimmer(time_df, thresh):
    cur = None
    for i, item in time_df.iteritems():
        if (cur is None) or (item - cur >= thresh):
            yield i
            cur = item

In [4]:
# Create a string format for the Day column
def dayConverter(s):
    # Set date formats
    time_format = "%Y-%m-%d"

    # Convert from period to string
    converted = s.strftime(time_format)
    
    return converted

In [5]:
# Add distance to all activities
def distanceEnhancer(added,x):
    return x + added
    

In [6]:
# Transforms seconds to hours
def secToHours(d):
    # Create the rule
    seconds = d
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    
    #Convert all to string
    seconds = str(seconds)
    minutes = str(minutes)
    hours = str(hours)
    
    #Add 0(zeros) for single digit numbers
    if len(seconds)==1:
        seconds = '0'+seconds
    if len(minutes)==1:
        minutes = '0'+minutes
    if len(hours)==1:
        hours = '0'+hours
    return hours+':'+minutes+':'+seconds

## Modify the data

In [7]:
# Convert to act_StartDate to datetime
df['act_startDate'] = pd.to_datetime(df['act_startDate'])
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Keep only date as a period in a seperate column
df['day'] = pd.DatetimeIndex(df['act_startDate']).to_period('D')

# Convert days to strings
df['day'] = df['day'].apply(lambda x: dayConverter(x))

# Sort df by ascending day and timestamp
df = df.sort_values(by=['day','timestamp'],ascending=[True,True])

# Check main df
df.head()

,act_startDate,timestamp,act_id,act_name,altitude,distance,grade_smooth,heartrate,moving,time,velocity_smooth,lat,long,day
0,2017-03-27 06:52:47,2017-03-27 06:52:47,916298883,Morning Ride,8.0,0.0,3.2,NaN,False,0,0.0,55.674874,12.592714,2017-03-27
1,2017-03-27 06:52:47,2017-03-27 06:52:57,916298883,Morning Ride,8.0,4.4,5.1,NaN,True,10,0.4,55.674910,12.592685,2017-03-27
2,2017-03-27 06:52:47,2017-03-27 06:52:59,916298883,Morning Ride,8.3,9.3,5.4,NaN,True,12,0.8,55.674938,12.592625,2017-03-27
3,2017-03-27 06:52:47,2017-03-27 06:53:01,916298883,Morning Ride,8.8,15.6,6.3,NaN,True,14,1.1,55.674979,12.592555,2017-03-27
4,2017-03-27 06:52:47,2017-03-27 06:53:02,916298883,Morning Ride,9.0,18.6,6.0,NaN,True,15,2.8,55.674997,12.592519,2017-03-27


## Create a day number flag for keeping track of the trip days    
## Create an iteration number flag for keeping track of activities per day

In [8]:
# Create helper dataframe with unique days
df_helper = df.groupby(by=['day','act_id']).count().iloc[:,0].reset_index().filter(items=['day','act_id'])

# Create columns for iter_no(per activity) and day_no(per day)
days = list(set(df_helper['day']))
days.sort()
day_no = list()
iter_no = list()
for index,day in enumerate(days):
    counter=1
    for dfday in df_helper['day']:
        if dfday == day:
            iter_no.append(counter)
            day_no.append(index+1)
            counter+=1

df_helper['day_no'] = pd.Series(day_no).values
df_helper['iter_no'] = pd.Series(iter_no).values      

df_helper

,day,act_id,day_no,iter_no
0,2017-03-27,916298883,1,1
1,2017-03-27,916806776,1,2
2,2017-04-01,923496071,2,1
3,2017-04-03,926539428,3,1
4,2017-04-06,930641527,4,1
5,2017-04-07,931935227,5,1
6,2017-04-11,937765185,6,1
7,2017-05-01,966163233,7,1
8,2017-05-01,966417205,7,2


In [9]:
# Merge flagger with the main dataframe
df = pd.merge(df,df_helper,on=['day','act_id'])

# Check main df
df.head()

,act_startDate,timestamp,act_id,act_name,altitude,distance,grade_smooth,heartrate,moving,time,velocity_smooth,lat,long,day,day_no,iter_no
0,2017-03-27 06:52:47,2017-03-27 06:52:47,916298883,Morning Ride,8.0,0.0,3.2,NaN,False,0,0.0,55.674874,12.592714,2017-03-27,1,1
1,2017-03-27 06:52:47,2017-03-27 06:52:57,916298883,Morning Ride,8.0,4.4,5.1,NaN,True,10,0.4,55.674910,12.592685,2017-03-27,1,1
2,2017-03-27 06:52:47,2017-03-27 06:52:59,916298883,Morning Ride,8.3,9.3,5.4,NaN,True,12,0.8,55.674938,12.592625,2017-03-27,1,1
3,2017-03-27 06:52:47,2017-03-27 06:53:01,916298883,Morning Ride,8.8,15.6,6.3,NaN,True,14,1.1,55.674979,12.592555,2017-03-27,1,1
4,2017-03-27 06:52:47,2017-03-27 06:53:02,916298883,Morning Ride,9.0,18.6,6.0,NaN,True,15,2.8,55.674997,12.592519,2017-03-27,1,1


## Create cumulative distance/time when changing activity

Keep the last row of each iteration of the same day and add its number to all the rest of the next day and do the same for the next activity

In [10]:
# Transform distance to cumulative distance when changing activity (per day)
for day in days:
    act_day = list(df_helper[df_helper['day']==day]['act_id'])
    if len(act_day)>1:
        last_dist = []
        last_time = []
        for i,act in enumerate(act_day):
            adding_dist = float(df[(df['day']==day) & (df['act_id']==act)]['distance'].iloc[-1:]) # +1
            adding_time = int(df[(df['day']==day) & (df['act_id']==act)]['time'].iloc[-1:]) # +1 if problem with division
            if i == 0:
                last_dist.append(adding_dist)
                last_time.append(adding_time)
            else:
                adding_dist_plus = adding_dist+last_dist[i-1]
                adding_time_plus = adding_time+last_time[i-1]
                last_dist.append(adding_dist_plus)
                last_time.append(adding_time_plus)
                df.loc[(df['day']==day) & (df['act_id']==act),'distance'] = df[(df['day']==day) & (df['act_id']==act)]['distance'].apply(
                    lambda x: distanceEnhancer(last_dist[i-1],x))
                df.loc[(df['day']==day) & (df['act_id']==act),'time'] = df[(df['day']==day) & (df['act_id']==act)]['time'].apply(
                    lambda x: distanceEnhancer(last_time[i-1],x))
        
#new = df.groupby(by=['day','act_id']).count().iloc[:,0].reset_index().filter(items=['day','act_id'])

## Create total elevation per day 

Create total elevation per day by summing the absolute difference between each consecutive point way before trimming.

In [11]:
# Create total elevation per day dataframe
result = {}
for day in days:
    temp_alt_df = df[df['day'] == day]['altitude']
    deltas = []
    for i in range(len(temp_alt_df)):
        if i>0:
            delta = abs(temp_alt_df.iloc[i]-temp_alt_df.iloc[i-1])
            deltas.append(delta)
    result[day] = sum(deltas)
    
# Create dataframe from result dictionary
alt_ttl_df = pd.DataFrame(result.items(), columns=['day', 'ttl_elevation'])
alt_ttl_df

,day,ttl_elevation
0,2017-04-06,271.6
1,2017-04-07,819.0
2,2017-04-11,242.5
3,2017-04-03,455.3
4,2017-03-27,520.1
5,2017-04-01,233.0
6,2017-05-01,805.0


## Create a speed column km/h

In [12]:
# Create an empty dataframe
speed_df = pd.DataFrame(columns = ['time','distance','speed'])

# Calculate the speed by measuring the m/s times 3.6 for km/h
for day in days:
    temp_speed_df = df[df['day'] == day].filter(items = ['time','distance'])
    speed_list = []
    for i in range(len(temp_speed_df)):
        if i==0:
            speed_list.append(0)
        else:
            dist_delta = float(temp_speed_df['distance'].iloc[i]-temp_speed_df['distance'].iloc[i-1])
            time_delta = int(temp_speed_df['time'].iloc[i]-temp_speed_df['time'].iloc[i-1])
            if time_delta == 0: # handles division error
                speed = speed_list[i-1]
                speed_list.append(speed)
            else:
                speed = (dist_delta/time_delta) * 3.6
                speed_list.append(speed)
    # Add new speed column to temp_speed_df
    temp_speed_df['speed'] = np.asarray(speed_list)
    speed_df = pd.concat([temp_speed_df,speed_df])

Check if the lengths match each other

In [13]:
print len(speed_df)
print len(df)

10416
10416


In [14]:
# Merge speed_df with the main dataframe on indexes
df = df.join(speed_df['speed'], how='outer')

Make sure that the speed makes sense

In [15]:
df[df['speed']>100]

,act_startDate,timestamp,act_id,act_name,altitude,distance,grade_smooth,heartrate,moving,time,velocity_smooth,lat,long,day,day_no,iter_no,speed
10407,2017-05-01 16:38:38,2017-05-01 17:04:28,966417205,Evening Ride,104.8,20969.2,6.3,138.0,True,2994,2.5,37.995915,23.740911,2017-05-01,7,2,222.6


In [16]:
df.iloc[10405:10409]

,act_startDate,timestamp,act_id,act_name,altitude,distance,grade_smooth,heartrate,moving,time,velocity_smooth,lat,long,day,day_no,iter_no,speed
10405,2017-05-01 16:38:38,2017-05-01 17:04:24,966417205,Evening Ride,102.4,20779.6,1.4,138.0,True,2990,2.1,37.994227,23.741208,2017-05-01,7,2,5.04
10406,2017-05-01 16:38:38,2017-05-01 17:04:25,966417205,Evening Ride,102.5,20783.7,6.3,138.0,True,2991,2.5,37.994264,23.741202,2017-05-01,7,2,14.76
10407,2017-05-01 16:38:38,2017-05-01 17:04:28,966417205,Evening Ride,104.8,20969.2,6.3,138.0,True,2994,2.5,37.995915,23.740911,2017-05-01,7,2,222.60
10408,2017-05-01 16:38:38,2017-05-01 17:04:43,966417205,Evening Ride,115.7,20988.2,7.5,138.0,True,3009,11.4,37.996086,23.740910,2017-05-01,7,2,4.56


In [17]:
# Remove those lines with extremely high speed. There must be inaccurate tracking
df = df[df['speed']<100]

## Create max and average speed per day and heartbeat

In [23]:
# Create table with avg_speed
avg_speed_df = df.groupby('day').mean().reset_index().sort_values(by='day', ascending=1).filter(items=['day','speed'])

# Change column name
avg_speed_df.rename(columns={'speed': 'avg_speed'}, inplace=True)

# Create table with avg_speed
max_speed_df = df.groupby('day').max().reset_index().sort_values(by='day', ascending=1).filter(items=['day','speed'])

# Change column name
max_speed_df.rename(columns={'speed': 'max_speed'}, inplace=True)

# Merge two dfs
ttl_speed_df = pd.merge(avg_speed_df,max_speed_df,on='day')

# Create table with avg_speed
avg_heartbeat_df = df.groupby('day').mean().reset_index().sort_values(by='day', ascending=1).filter(items=['day','heartrate'])

# Change column name
avg_heartbeat_df.rename(columns={'heartrate': 'avg_heartrate'}, inplace=True)

# Merge two dfs
ttl_speed_df = pd.merge(ttl_speed_df,avg_heartbeat_df,on='day')

ttl_speed_df

,day,avg_speed,max_speed,avg_heartrate
0,2017-03-27,12.627898,52.92,NaN
1,2017-04-01,9.832409,18.00,NaN
2,2017-04-03,10.572112,37.08,NaN
3,2017-04-06,10.808719,18.36,NaN
4,2017-04-07,10.581511,41.40,NaN
5,2017-04-11,9.665677,48.60,134.161850
6,2017-05-01,26.873455,61.02,126.394024


## Create cumulative distance in string format (hours:minutes:seconds)

In [19]:
# Create column with converted seconds to hours:minutes:seconds format (string)
df['time_form'] = df['time'].apply(lambda x: secToHours(x))

## Keep only selected columns

In [20]:
df = df.filter(items=['day','act_startDate','timestamp','day_no','iter_no','altitude','distance','heartrate','time','time_form','speed','long','lat'])

In [21]:
df

,day,act_startDate,timestamp,day_no,iter_no,altitude,distance,heartrate,time,time_form,speed,long,lat
0,2017-03-27,2017-03-27 06:52:47,2017-03-27 06:52:47,1,1,8.0,0.0,NaN,0,00:00:00,0.000,12.592714,55.674874
1,2017-03-27,2017-03-27 06:52:47,2017-03-27 06:52:57,1,1,8.0,4.4,NaN,10,00:00:10,1.584,12.592685,55.674910
2,2017-03-27,2017-03-27 06:52:47,2017-03-27 06:52:59,1,1,8.3,9.3,NaN,12,00:00:12,8.820,12.592625,55.674938
3,2017-03-27,2017-03-27 06:52:47,2017-03-27 06:53:01,1,1,8.8,15.6,NaN,14,00:00:14,11.340,12.592555,55.674979
4,2017-03-27,2017-03-27 06:52:47,2017-03-27 06:53:02,1,1,9.0,18.6,NaN,15,00:00:15,10.800,12.592519,55.674997
5,2017-03-27,2017-03-27 06:52:47,2017-03-27 06:53:03,1,1,9.1,21.8,NaN,16,00:00:16,11.520,12.592488,55.675019
6,2017-03-27,2017-03-27 06:52:47,2017-03-27 06:53:04,1,1,9.3,26.1,NaN,17,00:00:17,15.480,12.592440,55.675047
7,2017-03-27,2017-03-27 06:52:47,2017-03-27 06:53:05,1,1,9.3,30.1,NaN,18,00:00:18,14.400,12.592395,55.675072
8,2017-03-27,2017-03-27 06:52:47,2017-03-27 06:53:06,1,1,9.3,36.5,NaN,19,00:00:19,23.040,12.592342,55.675122
9,2017-03-27,2017-03-27 06:52:47,2017-03-27 06:53:07,1,1,9.2,41.3,NaN,20,00:00:20,17.280,12.592270,55.675138


## Extract data to a csv to check

In [38]:
df.to_csv('test.csv', index=False)

## Trim data points if they are too many to be handled by D3

In [28]:
df_trim = df.loc[list(trimmer(df['timestamp'], pd.to_timedelta(30, 's')))]

In [29]:
# Check if the trimming is correct
df_trim.head()

,act_startDate,timestamp,act_id,act_name,altitude,distance,grade_smooth,heartrate,moving,time,velocity_smooth,lat,long,day,day_no,iter_no
0,2017-03-27 06:52:47,2017-03-27 06:52:47,916298883,Morning Ride,8.0,0.0,3.2,NaN,False,0,0.0,55.674874,12.592714,2017-03-27,1,1
18,2017-03-27 06:52:47,2017-03-27 06:53:17,916298883,Morning Ride,7.9,88.9,0.0,NaN,True,30,4.1,55.675395,12.592052,2017-03-27,1,1
48,2017-03-27 06:52:47,2017-03-27 06:53:47,916298883,Morning Ride,3.9,283.2,0.9,NaN,True,60,7.0,55.676493,12.594387,2017-03-27,1,1
65,2017-03-27 06:52:47,2017-03-27 06:54:17,916298883,Morning Ride,1.0,500.0,1.5,NaN,True,90,6.7,55.677439,12.597199,2017-03-27,1,1
90,2017-03-27 06:52:47,2017-03-27 06:54:47,916298883,Morning Ride,6.7,650.3,4.2,NaN,True,120,6.6,55.678280,12.596714,2017-03-27,1,1
